# Eerste verkenning en validatie van de observation details. 
Verkennen van de data en uitvoeren van een eerste ruwe cleaning. Cleaning betreft het verwijderen van locatie_id's die niet meer bestaan en onbekende oppervlaktes allemaal op dezelfde manier weergeven (string None)</br>
Het resultaat wordt geplaatst in de sectie 'cleaned'

In [49]:
import pandas as pd
import os

In [33]:
folder_path_clean = "./scraped_data/cleaned/"

In [34]:
# set the max columns to none
pd.set_option('display.max_columns', None)
# set the max columns to none
pd.set_option('display.max_rows', None)


## Verkenning van de data


In [35]:
# file path
location_details = f'scraped_data/location_details_complete.csv'


# Load the data
df = pd.read_csv(location_details, skipinitialspace=True, index_col=1)
df.head()

,location_name,province,province_id,municipality,municipality_id,location_type,page,area
location_id,,,,,,,,
241620,A Fladjon,Liège,22.0,Ans (entité),43743.0,Gebied,1,4.780 m2
252470,A la Creux (réserve naturelle),Luxembourg,26.0,Léglise (entité),43953.0,Gebied,1,"18,13 ha"
241688,A St-Jacques,Namur,24.0,Somme-Leuze (entité),43891.0,Gebied,1,"30,22 ha"
27478,Aaigem (Dg),Oost-Vlaanderen,16.0,Erpe-Mere,24153.0,Deelgemeente,1,"7,41 km2"
31885,Aalbeke - Allartpark,West-Vlaanderen,15.0,Aalbeke (Dg),24033.0,Gebied,1,"2,16 ha"


In [36]:
df.index 

Index([241620, 252470, 241688,  27478,  31885,  71763,  83947,  24033, 724399,
        83946,
       ...
        29903, 191328,  28347, 191310,  29933,  29905, 191315,  29906,  29904,
        28705],
      dtype='int64', name='location_id', length=14040)

In [37]:
df.columns

Index(['location_name', 'province', 'province_id', 'municipality',
       'municipality_id', 'location_type', 'page', 'area'],
      dtype='object')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14040 entries, 241620 to 28705
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location_name    14040 non-null  object 
 1   province         14040 non-null  object 
 2   province_id      14039 non-null  float64
 3   municipality     14040 non-null  object 
 4   municipality_id  13646 non-null  float64
 5   location_type    14040 non-null  object 
 6   page             14040 non-null  int64  
 7   area             14038 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 987.2+ KB


In [39]:
stats = df.describe(include='all')
stats

,location_name,province,province_id,municipality,municipality_id,location_type,page,area
count,14040,14040,14039.000000,14040,13646.000000,14040,14040.000000,14038
unique,13994,12,NaN,716,NaN,4,NaN,7153
top,Ancienne Carrière de l'Eau Bleue (La Bruyère),Oost-Vlaanderen,NaN,—,NaN,Gebied,NaN,onbekend
freq,2,2270,NaN,394,NaN,10895,NaN,72
mean,NaN,NaN,19.169243,NaN,33974.214568,NaN,281.300214,NaN
std,NaN,NaN,3.881507,NaN,15465.777904,NaN,162.125842,NaN
min,NaN,NaN,14.000000,NaN,23089.000000,NaN,1.000000,NaN
25%,NaN,NaN,16.000000,NaN,23441.000000,NaN,141.000000,NaN
50%,NaN,NaN,17.000000,NaN,24169.500000,NaN,281.000000,NaN
75%,NaN,NaN,23.000000,NaN,43850.000000,NaN,422.000000,NaN


In [40]:
df.isnull().sum()

location_name        0
province             0
province_id          1
municipality         0
municipality_id    394
location_type        0
page                 0
area                 2
dtype: int64

# First cleaning

## Ongeldige area

In [41]:
df[df['area'].isnull()]

,location_name,province,province_id,municipality,municipality_id,location_type,page,area
location_id,,,,,,,,
241323,Carrière des Douze Bonniers,Hainaut,23.0,Gerpinnes (entité),43896.0,Gebied,101,NaN
189774,Gentbrugge - Meersen,Oost-Vlaanderen,16.0,Gent,24108.0,Gebied,176,NaN


De locations (location_id) met area Nan  

Als we deze verifiëren op de website (bv. https://waarnemingen.be/locations/241323/), dan zien we dat deze inderdaad niet bestaan.  
Een verklaring hiervoor is dat deze nog wel bestonden toen we de general data ophaalden, maar nadien verwijderd zijn geweest. We houden deze wel aanwezig in onze dataset, vermits deze misschien nog gekoppeld zijn aan waarnemingen die we ooit hebben opgehaald.

# Gebruikte waarden en hun frequentie checken

#### conclusie
Area heeft niet altijd een waarde. Soms is deze 'onbekend', als deze zo wordt meegegeven in waarnemingen.be.
Soms is deze 'NaN' als de locatie niet meer bestond op waarnemingen.be
We zetten deze nu allemaal op waarde None, zodat dit veld altijd als een string object kan geïnterpreteerd worden.




In [ ]:
df[df['area'].notnull() & ~(df['area'].astype(str).str.contains("m2")) & ~(df['area'].astype(str).str.contains("ha"))].head()

,location_name,province,province_id,municipality,municipality_id,location_type,page,area
location_id,,,,,,,,
599538,Chateau de Moulins Anhée,Namur,24.0,Anhée (entité),43903.0,project-Gebied,106,onbekend
610189,IPA_132 Wilder,Bruxelles / Brussel,19.0,Sint-Agatha-Berchem/Berchem-Sainte-Agathe,23161.0,project-Gebied,228,onbekend
610194,IPA_133 Laarbeekbos 1,Bruxelles / Brussel,19.0,Jette,23163.0,project-Gebied,228,onbekend
610196,IPA_134 Laarbeebkbos 2,Bruxelles / Brussel,19.0,Jette,23163.0,project-Gebied,228,onbekend
610195,IPA_135 Laarbeekbos 3,Bruxelles / Brussel,19.0,Jette,23163.0,project-Gebied,228,onbekend
610197,IPA_213 Bois de Dieleghem,Bruxelles / Brussel,19.0,Jette,23163.0,project-Gebied,228,onbekend
610188,IPA_401 Scheutbos,Bruxelles / Brussel,19.0,Molenbeek-Saint-Jean/Sint-Jans-Molenbeek,23158.0,project-Gebied,228,onbekend
610185,IPA_402,Bruxelles / Brussel,19.0,Anderlecht,23157.0,project-Gebied,228,onbekend
610193,IPA_500 Poelbos,Bruxelles / Brussel,19.0,Jette,23163.0,project-Gebied,228,onbekend


In [45]:
df[df['area'].notnull() & ~(df['area'].astype(str).str.contains("m2")) & ~(df['area'].astype(str).str.contains("ha"))].shape[0]

72

In [47]:
# onbekende values van area op None zetten
df_first_cleaning = df.copy()
df_first_cleaning['area'] = df_first_cleaning['area'].apply(
    lambda x: None if pd.isnull(x) or (isinstance(x, str) and x.lower() == "onbekend") else x)


## Saven van de gecleande data

In [50]:
# Save the cleaned DataFrame to a new CSV file
df_first_cleaning.to_csv(os.path.join(folder_path_clean, f"location_details_clean.csv"), index=True)